In [ ]:
# visualizePRISM.ipynb
# Licensed under the MIT License
# © Author: Sarah Mokhtar

In [ ]:
import os, sys
from pathlib import Path

def _find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for parent in (p, *p.parents):
        if (parent / ".git").exists() or (parent / "UPSTREAM.md").exists() or (parent / "requirements.txt").exists():
            return parent
    return start.resolve().parents[2]

HERE = Path.cwd()
ROOT = _find_repo_root(HERE)

sys.path.insert(0, str(ROOT / "src"))
sys.path.insert(0, str(ROOT / "third_party"))

from prism.viz.viz_utils import *

##### Directory definition

In [ ]:
cur_dir = '' # Add folder location of dataset subsample

In [ ]:
b_name = 'building_345'
sample_dir = os.path.join(cur_dir,b_name)

##### Inspect Directory Contents

In [ ]:
raw_dir = os.path.join(sample_dir, 'raw')
proc_dir = os.path.join(sample_dir, 'proc')

In [ ]:
print(os.listdir(raw_dir))

In [ ]:
print('Directories for raw data:')
for root, dirs, files in os.walk(raw_dir):
    for dir in dirs:
        print(dir)

In [ ]:
print('All files for raw data:')
for root, dirs, files in os.walk(raw_dir):
    for file in files:
        print(file)

In [ ]:
print('All directories for processed CFD data:')
for root, dirs, files in os.walk(proc_dir):
    for dir in dirs:
        print(dir)

In [ ]:
print('All files for processed CFD data:')
for root, dirs, files in os.walk(proc_dir):
    for file in files:
        parent_dir = os.path.basename(root)
        print(f"{parent_dir}/{file}")

In [ ]:
for root, dirs, files in os.walk(proc_dir):
    for dir in dirs:
        print(dir)

##### Visualize Geometry

In [ ]:
b_mesh = trimesh.load_mesh(os.path.join(raw_dir,'geometry','explicit',b_name+'.obj'), skip_materials=True)
bbox = np.array([[-260.,  260.],[ -10.,  510.], [-260.,  260.]])

In [ ]:
visualize_mesh(b_mesh.vertices, b_mesh.faces)

###### Point cloud

In [ ]:
pc = np.load(os.path.join(raw_dir,'geometry','implicit','zeroSurface',b_name+'_zeroSurface.npz'))['alls']
xyz = pc[:,0:3]
nor = pc[:,3:]

In [ ]:
n_points = 5000
rand_idx = np.random.randint(0,xyz.shape[0],n_points)
norm_mesh = normalize_mesh(b_mesh, bbox)
normals_unit = nor / np.linalg.norm(nor, axis=1, keepdims=True)
theta = np.arctan2(normals_unit[:,1], normals_unit[:,0])
values = (theta + np.pi) / (2 * np.pi)

In [ ]:
plot_points_mesh(norm_mesh, xyz[rand_idx], values[rand_idx].flatten(), bbox=[[0,1],[0,1],[0,1]]).show()

###### SDF

In [ ]:
samp_types = ['rejection','surface','uniform','combined/rej_100_sur_100_uni_100_zer_100','combined/rej_100_sur_100_uni_10_zer_100']
sel_stype = 'rejection'
sel_sID = samp_types.index(sel_stype)

In [ ]:
p_type = 'pos'   # or 'zero' or 'neg'
sdf_byT = np.load(os.path.join(raw_dir,'geometry','implicit',sel_stype,b_name+'_'+sel_stype+'.npz'))[p_type]

In [ ]:
xyz = sdf_byT[:,0:3]
sdf_v = sdf_byT[:,3:]

In [ ]:
n_points = 5000
rand_idx = np.random.randint(0,xyz.shape[0],n_points)
norm_mesh = normalize_mesh(b_mesh, bbox)
values = sdf_v

In [ ]:
plot_points_mesh(norm_mesh, xyz[rand_idx], values[rand_idx].flatten(), bbox=[[0,1],[0,1],[0,1]], cmin_v=0, cmax_v=0.1).show()

##### Visualize Performance

###### CFD

In [ ]:
av_windD = np.array([int(d.split('_')[2]) for d in os.listdir(proc_dir) if os.path.isdir(os.path.join(proc_dir, d))])
av_windD

In [ ]:
sel_windD = 0

In [ ]:
path_srf = proc_dir
path_byO = os.path.join(proc_dir, b_name + '_' + str(sel_windD))

In [ ]:
y = np.load(os.path.join(path_byO,'y.npy'))
vel = y[:,0:3]
sp = np.linalg.norm(vel, axis=1)
pres = y[:,4:]
print('vel.shape',vel.shape,'sp.shape',sp.shape,'pres.shape',pres.shape)

In [ ]:
x = np.load(os.path.join(path_srf,'x.npy'))
pos = x[:,0:3]
sdf = x[:,3:4]
nor = x[:,4:]
print('pos.shape',pos.shape,'sdf.shape',sdf.shape,'nor.shape',nor.shape)

In [ ]:
surf = np.load(os.path.join(path_srf,'surf.npy'))
print('surf.shape',surf.shape)

In [ ]:
plot_srf_pressure(pos, surf, pres)

In [ ]:
visualize_velocity_field(pos, sp)

In [ ]:
visualize_velocity_in_domain(pos, vel, n_pts=20000)

###### SVF

In [ ]:
b_mesh = trimesh.load_mesh(os.path.join(raw_dir,'geometry','explicit',b_name+'.obj'), skip_materials=True)
bbox = np.array([[-260.,  260.],[ -10.,  510.], [-260.,  260.]])
srf_pts = pd.read_csv(os.path.join(raw_dir,'geometry','perf_grid',b_name+'_srfP_20.csv'), header=None).values.round(3)[:,[0,2,1]]
srf_pts[:,2] = srf_pts[:,2]*-1
grd_pts = pd.read_csv(os.path.join(raw_dir,'geometry','perf_grid','XYgrid_256_30.csv'), header=None).values.round(3)[:,[0,2,1]]
grd_pts[:,1] = 1.5
grd_SVF = np.load(os.path.join(raw_dir,'performance','svfbypat',b_name+'_XYgrid_256_30_15_svfbypat.npy'))
srf_SVF = np.load(os.path.join(raw_dir,'performance','svfbypat',b_name+'_srf_20_svfbypat.npy'))
print('srf_pts',srf_pts.shape,'grd_pts',grd_pts.shape,'grd_SVF',grd_SVF.shape,'srf_SVF',srf_SVF.shape)

In [ ]:
plot_points_mesh(b_mesh, srf_pts, np.sum(srf_SVF,axis=1)/577, bbox, cmin_v = 0, cmax_v = 1).show()

In [ ]:
plot_points_mesh(b_mesh, grd_pts, np.sum(grd_SVF,axis=1)/577, bbox, cmin_v = 0, cmax_v = 1).show()

In [ ]:
# Fir different sky patches
sky_pat = 6
plot_points_mesh(b_mesh, grd_pts, grd_SVF[:,sky_pat], bbox, cmin_v = 0, cmax_v = 1).show()

In [ ]:
# Fir different sky patches
sky_pat = 200
plot_points_mesh(b_mesh, grd_pts, grd_SVF[:,sky_pat], bbox, cmin_v = 0, cmax_v = 1).show()